# soilVisualization.ipynb

After [aggregating the soil data](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions/blob/main/src/importSoil/soilaggregation.ipynb) and [aggregating the ergot data](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions/blob/main/src/Ergot/aggregateErgot.ipynb) the following script can be used to visualize the data

##### Output graphs:
- Correlation plot
- Pair plots

In [ ]:
import matplotlib.pyplot as plt  # type: ignore
from dotenv import load_dotenv
import sqlalchemy as sq
import seaborn as sns  # type: ignore
import pandas as pd
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
SOIL_TABLE = "agg_soil_data" # name of the table we'd like to pull the soil data from
ERGOT_TABLE = "agg_ergot_sample" # name of the table we'd like to pull the ergot data from


# Load the database connection environment variables located in the docker folder
load_dotenv("../docker/.env")
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

Purpose:  
Self contained data retrieval for the soil/ergot visualization

Psuedocode:  
- Load the environment database variables
- Connect to the database
- Create the soil data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) 
- Create the ergot data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- Close the database connection

In [ ]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

# Pull the soil data
soilQuery = sq.text(f"SELECT * FROM public.{SOIL_TABLE};")
soilDf = pd.read_sql_query(soilQuery, conn)

# Pull the ergot data
ergotQuery = sq.text(f"SELECT * FROM public.{ERGOT_TABLE}")
ergotDF = pd.read_sql_query(ergotQuery, conn)

db.cleanup()

Purpose:  
Merge the soil 

Psuedocode:  
- Load the environment database variables
- Connect to the database
- Create the soil data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) 
- Create the ergot data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- Close the database connection

In [ ]:
final_df = soilDf.merge(ergotDF, on=["district"])
final_df.drop(columns=["district", "year"], inplace=True)

Purpose:  
Self contained data retrieval for the soil/ergot visualization

Psuedocode:  
- Load the environment database variables
- Connect to the database
- Create the soil data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) 
- Create the ergot data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- Close the database connection

In [ ]:
corr = final_df.corr()

Purpose:  
Self contained data retrieval for the soil/ergot visualization

Psuedocode:  
- Load the environment database variables
- Connect to the database
- Create the soil data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) 
- Create the ergot data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- Close the database connection

In [ ]:
sns.set_theme(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(
    corr,
    mask=mask,
    cmap=cmap,
    vmax=1,
    vmin=-1,
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.5},
)

Purpose:  
Self contained data retrieval for the soil/ergot visualization

Psuedocode:  
- Load the environment database variables
- Connect to the database
- Create the soil data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) 
- Create the ergot data SQL query
- [Load the data from the database directly into a DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html)
- Close the database connection

In [ ]:
sns.pairplot(final_df[["has_ergot", "sum_severity", "avg_calcium_ph", "avg_proj_ph"]])
plt.show()

sns.pairplot(
    final_df[["has_ergot", "sum_severity", "avg_total_sand", "avg_water_reten_10"]]
)
plt.show()